In [1]:
import numpy as np
import cv2 as cv
import mediapipe as mp
import math
from playsound import playsound
import time
import multiprocessing

In [2]:
mp_face_mesh = mp.solutions.face_mesh
distances = []
focii = []
message = None
colour = None

In [3]:
LEFT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]
RIGHT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_IRIS = [469, 470, 471, 472]
RIGHT_IRIS = [474, 475, 476, 477]
CENTERS = [468, 473]

In [4]:
def install_commands():
    texts = ['Go right', 'Go left', 'Go up', 'Go down', 'Get closer', 'Stay in Square']
    language = 'en'
    for text in texts:
        text_obj = gTTS(text = text, lang = language, slow = False)
        text_obj.save(f'{text}.mp3')

In [5]:
# install_commands()

In [6]:
def mediapipe_detection(image, face):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = face.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, results

In [7]:
def get_distance(coord1, coord2):
    
    irises_dist_in_pixels = math.dist(coord1, coord2)
    irises_dist_in_rw = 7
    face_dist_in_rw = 80
    focal_length = (irises_dist_in_pixels * face_dist_in_rw) / irises_dist_in_rw
    focii.append(focal_length)
    
    
    #focal length for distance = 70cm is 611cm
    #focal length for distance = 60cm is 618cm
    #focal length for distance = 50cm is 602cm
    #focal length for distance = 40cm is 610cm
    #average focal length = 610.25cm
    
    avg_focal_length = 610
    actual_distance = (avg_focal_length * irises_dist_in_rw) / irises_dist_in_pixels
    distances.append(actual_distance)

In [8]:
def part_detection(part, facial_landmarks, image_shape):
    idx_to_coordinates = {}
    img_height, img_width, _ = image_shape
    for idx, landmark in enumerate(part):
        coord = facial_landmarks.landmark[landmark]
        
        p_x = min(math.floor(coord.x * img_width), img_width - 1)
        p_y = min(math.floor(coord.y * img_height), img_height - 1)
        landmark_coord = (p_x, p_y)
        
        if landmark_coord:
            idx_to_coordinates[idx] = landmark_coord
    return idx_to_coordinates

In [9]:
def display_part(image, coords, color, show_landmarks = False, center_coords = False):
    points = np.zeros(shape = (len(coords), 2))
    for idx, landmark_px in coords.items():
        points[idx] = landmark_px
        if show_landmarks == True:
            cv.circle(image, landmark_px, 1, (255, 0, 0), 2)
    points = points.astype(np.int32)
    
    #Only for center coordinates
    if center_coords == False:
        cv.polylines(image, [points], True, color, 2)

    

In [10]:
def create_bool_array(x1, y1, x2, y2):
    arr = []
    for i in range(x1, x2):
        for j in range(y1, y2):
            arr.append((i,j))
    return arr

In [11]:
def display_regions(image):
    #creating frame lines
    cv.line(image, (0, 240), (640, 240), (255,0,255), 1)
    cv.line(image, (320, 0), (320, 480), (255,0,255), 1)
    
    

In [30]:
def play_sound(count):
    msgs = ['Go left', 'Go right', 'Go up', 'Go down', 'Stay in Square', 'Get closer']
    init_time = time.time()
    current_time = 0
    current2_time = time.time()
    print("hello")
    while count.value != -1:
        
        current_time = time.time()
        if current_time - init_time > 3:
            playsound("{}.mp3".format(msgs[count.value]))
            init_time = time.time()
            print("sound:", msgs[count.value])


In [36]:
def correct_position(image, center_coords, count):
    
    #sound messages
    msgs = ['Go left', 'Go right', 'Go up', 'Go down', 'Stay in Square', 'Get closer']
    
    
    #getting the distances
    ideal_dist = 50
    actual_dist = distances[-1]
    
    #message to be displayed
    
    
    #left rectangle
    left_x1, left_y1 = 220, 210
    left_x2, left_y2 = 300, 270
    
    #right_rectangle
    right_x1, right_y1 = 340, 210
    right_x2, right_y2 = 420, 270
    
    left_center_x = center_coords[0][0]
    left_center_y = center_coords[0][1]
    
    right_center_x = center_coords[1][0]
    right_center_y = center_coords[1][1]
    
    arr1 = create_bool_array(left_x1, left_y1, left_x2, left_y2)
    arr2 = create_bool_array(right_x1, right_y1, right_x2, right_y2)
    
    if  (left_center_x, left_center_y) in arr1 and (right_center_x, right_center_y) in arr2:
        #message = "Stay in Square"
        count.value = 4
        colour = (0,255,0)
        
        if actual_dist > ideal_dist:
            #message = "Get closer"
            count.value = 5
            colour = (0, 0, 255)
        
        cv.rectangle(image, (left_x1, left_y1), (left_x2, left_y2), colour, 1)
        cv.rectangle(image, (right_x1, right_y1), (right_x2, right_y2), colour, 1)
    
    else:
        colour = (0,0,255)
        cv.rectangle(image, (left_x1, left_y1), (left_x2, left_y2), colour, 1)
        cv.rectangle(image, (right_x1, right_y1), (right_x2, right_y2), colour, 1)
        
        if left_center_x < left_x1 or right_center_x < right_x1:
            #message = "Go right"
            count.value = 1
                        
        elif left_center_x > left_x2 or right_center_x > right_x2:
            #message = "Go left"
            count.value = 0
            
        elif left_center_y < left_y1 or right_center_y < right_y1:
            #message = "Go down"
            count.value = 3
            
        elif left_center_y > left_y2 or right_center_y > right_y2:
            # message = "Go up"
            count.value = 2   

    cv.putText(image, msgs[count.value], (200, 100), cv.FONT_HERSHEY_SIMPLEX, 1, colour, 2, cv.LINE_AA)

In [37]:
# def correct_position(image, facial_landmarks, center, center_coords):
#     print(center_coords)

In [38]:
def capture(cap, count):
#     cap = cv.VideoCapture(0)
    message = None
    initial_time = time.time()
    with mp_face_mesh.FaceMesh(
        max_num_faces = 1,
        refine_landmarks = True,
        min_detection_confidence = 0.5,
        min_tracking_confidence = 0.5
    ) as face_mesh:
        while True:
            ret, frame = cap.read()
            current_time = time.time()
            if not ret:
                print("Not ret")
                break


            image = cv.flip(frame, 1)
            image, results = mediapipe_detection(image, face_mesh)
            img_height, img_width, _ = image.shape

    #             print(image.shape)
            if results.multi_face_landmarks:
                for num, facial_landmarks in enumerate(results.multi_face_landmarks):

                    left_eye_coords = part_detection(LEFT_EYE, facial_landmarks, image.shape)
                    right_eye_coords = part_detection(RIGHT_EYE, facial_landmarks, image.shape)
    #                 left_iris_coords = part_detection(LEFT_IRIS, facial_landmarks, image.shape)
    #                 right_iris_coords = part_detection(RIGHT_IRIS, facial_landmarks, image.shape)
                    centers_coords = part_detection(CENTERS, facial_landmarks, image.shape)


                    display_part(image, left_eye_coords, (0,255,255), False)
                    display_part(image, right_eye_coords, (0,255,255), False)
    #                 display_part(image, left_iris_coords, (0,255,255), True)
    #                 display_part(image, right_iris_coords, (0,255,255), True)
                    display_part(image, centers_coords, (0,255,255), True, True)

            display_regions(image)
            get_distance(centers_coords[0], centers_coords[1])


            correct_position(image, centers_coords, count)

            cv.imshow("video feed", image)
            if cv.waitKey(5) & 0xff == ord('q'):
                count.value = -1
                break
    print(sum(distances) / len(distances))
    print(distances[-1])
    cap.release()
    cv.destroyAllWindows()

In [39]:
def main():
#     install_commands()
    cap = cv.VideoCapture(0)
    count = multiprocessing.Value('i')
    
    p1 = multiprocessing.Process(target = capture, args = (cap, count))
    p2 = multiprocessing.Process(target = play_sound, args = (count, ))
    
    p1.start()
    p2.start()
    
    p1.join()
    p2.join()
    
    print("finsihed session")

In [40]:
if __name__ == '__main__':
    main()

hello


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


sound: Go right
sound: Go down
sound: Stay in Square
sound: Go right
sound: Go left
sound: Go down
54.47134612615369
71.8688213257567
finsihed session
